In [29]:
import pandas as pd
import numpy as np

## Initial DF Setup

In [30]:
### Setting the size of the gage r&r
num_parts = 10 #input(How many parts ran?)
num_trials = 3 #input(How many trials per part?)
num_operators = 3 #input(How many operators?)

In [31]:
### Reading the data from the measuring system
df_raw = pd.read_csv('RawData.csv') # creating the variable df to present the data frame
df_raw.head(3), df_raw.shape

(       DateTime                              SerialNum CharName   X Pos  \
 0  06324_131250  EXT1C0070008018000277700000063332611A     C2_3 -48.127   
 1  06324_131250  EXT1C0070008018000277700000063332611A      C4A -99.079   
 2  06324_131250  EXT1C0070008018000277700000063332611A      C4B -99.059   
 
      Y Pos  Z Pos  TP Measure  Diameter  Length PassTP PassDiameter  \
 0   95.770    0.0       0.283     6.989  10.883   TRUE         TRUE   
 1  -12.134    0.0       0.079     7.896   0.000   TRUE        FALSE   
 2 -108.026    0.0       0.097     7.906   0.000   TRUE         TRUE   
 
   PassLength PassTot  
 0       TRUE    TRUE  
 1       TRUE   FALSE  
 2       TRUE    TRUE  ,
 (2790, 13))

In [32]:
### Simplify the serial number

# With a new column, extracting the simple serial number 
df_raw['SN'] = df_raw['SerialNum'].str[27:30]  # Indexing starts at 0, so the 28th character is at index 27

df_raw.head(1)


,DateTime,SerialNum,CharName,X Pos,Y Pos,Z Pos,TP Measure,Diameter,Length,PassTP,PassDiameter,PassLength,PassTot,SN
0,06324_131250,EXT1C0070008018000277700000063332611A,C2_3,-48.127,95.77,0.0,0.283,6.989,10.883,TRUE,TRUE,TRUE,TRUE,063


In [33]:
### Verify Data is Complete

# Verify unique serial numbers matches the number of parts required
if df_raw['SN'].unique().size == num_parts:
    print(f'Serial numbers match the number of parts required')
else:
    print(f'ERROR: numbers DO NOT match the number of parts required')

# Verify all measurments are present
#if df_raw['TP Measure'] == "UTM":
#    print(f'UTM Code: Not all measurements present, see index')



Serial numbers match the number of parts required


## Transforming True Position

### Working with one characteristic before iterating

In [34]:
# New data frame that only contains DateTime, CharName, and TP Measure
df_Char_TP = df_raw[['DateTime', 'SN', 'CharName', 'TP Measure']]
    # Keep the Datetime to esnure the order of measurements

# Filtering on a single characteristic, in this case C4A
df_single_char = df_Char_TP[df_Char_TP['CharName'] == 'C4A']

len(df_single_char)

# Verify the number of measurements matches the number of parts, operators, trials required
if len(df_single_char) == num_parts*num_trials*num_operators: # len() returns the number of rows
                                                            #Essentially equaling the size of the gage rr
    print('Number of measurements matches the gage r&r size')
else:
    print('ERROR: Number of measurements DOES NOT matches the gage r&r size')

df_single_char.head(3)


Number of measurements matches the gage r&r size


,DateTime,SN,CharName,TP Measure
1,06324_131250,063,C4A,0.079
32,06324_131539,064,C4A,0.418
63,06324_131716,211,C4A,0.441


In [35]:
### Transforming Dataframe

# Renaming the TP Measure column
df_single_char_trans = df_single_char.rename(columns={'TP Measure': 'C4A_TP'})
    # Pandas warning about using parameter inplace, need to set the DF equal to itself

# Dropping CharName
df_single_char_trans.drop(columns='CharName', inplace=True)

df_single_char_trans.head(3)


,DateTime,SN,C4A_TP
1,06324_131250,063,0.079
32,06324_131539,064,0.418
63,06324_131716,211,0.441


In [36]:
### Add the Operator
# Ensure measurements are in chronological order
df_single_char_trans = df_single_char_trans.sort_values('DateTime',ascending=True) 
    # .sort_values does not change in place

In [37]:
# Create Array of Operators
'''
Currently this assumes three operators will always be used. Need to modify the code to expand and 
contract with the number of the operators. 

Assumes based on timestamp that first operator ran all parts and trials, then the second operator, and so forth
'''

gage_rr_size = num_operators*num_parts*num_trials

operator_list = []
i = 0
while i < gage_rr_size:
    if i < gage_rr_size*(1/3): # First third is from Operator 1
        operator_list.append(1)
        i+=1
    elif i < gage_rr_size*(2/3): # Second third is from Operator 2 
        operator_list.append(2)
        i+=1
    elif i < gage_rr_size*(3/3): # Last third is from Operator 3
        operator_list.append(3)
        i+=1

## Tests
# print(operator_list)
# len(operator_list)
# operator_list.count(1)

In [38]:
# Combine Operator list and Char DF
df_single_char_trans['Operator'] = operator_list
df_single_char_trans.head()

,DateTime,SN,C4A_TP,Operator
1,06324_131250,063,0.079,1
32,06324_131539,064,0.418,1
63,06324_131716,211,0.441,1
94,06324_131856,075,0.441,1
125,06324_132033,214,0.309,1


In [39]:
### Add the trails
# Ensure data is properly ordered chronologically by SN and Operator
df_single_char_trans = df_single_char_trans.sort_values(by=['SN', 'DateTime'],ascending=[True, True])
df_single_char_trans.head(20)

,DateTime,SN,C4A_TP,Operator
1,06324_131250,063,0.079,1
559,06324_134613,063,0.089,1
652,06324_135842,063,0.068,1
1117,06324_142613,063,0.082,2
1458,06324_144658,063,0.059,2
1644,06324_151955,063,0.069,2
2047,06324_154344,063,0.073,3
2295,06324_160431,063,0.064,3
2698,06324_162736,063,0.088,3
32,06324_131539,064,0.418,1


In [40]:
# Generate trials array
'''
Assumes the trial number is three. This needs to be able to expand and contract base on the 
number of trials inputted
'''
# Need to repeat the trail number
array_to_repeat = np.array([1, 2, 3])

# Specify how many times to repeat
x = num_operators*num_parts

# Create the repeated array
repeated_array = np.tile(array_to_repeat, x)

## Tests
print(repeated_array)
len(repeated_array)

[1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1
 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2
 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3]


90

In [41]:
# Combine with Char DF
df_single_char_trans['Trial Num'] = repeated_array
df_single_char_trans

,DateTime,SN,C4A_TP,Operator,Trial Num
1,06324_131250,063,0.079,1,1
559,06324_134613,063,0.089,1,2
652,06324_135842,063,0.068,1,3
1117,06324_142613,063,0.082,2,1
1458,06324_144658,063,0.059,2,2
...,...,...,...,...,...
1396,06324_144321,216,0.419,2,2
1799,06324_152825,216,0.416,2,3
1954,06324_153821,216,0.410,3,1
2357,06324_160801,216,0.420,3,2


### Iterate over all charateristics to build miniTab dataframe

In [42]:
### Retreive all the chars
all_char = df_raw['CharName'].unique()
all_char

array(['C2_3', 'C4A', 'C4B', 'C4C', 'C4D', 'C4E', 'C4F', 'C4G', 'C4H',
       'C11', 'C13A', 'C13B', 'C17A', 'C17B', 'C18A', 'C18B', 'C26A',
       'C26B', 'C26C', 'C26D', 'C27A', 'C27B', 'C27C', 'C27D', 'C31A',
       'C37A', 'C50A', 'C31B', 'C37B', 'C50B', 'C52'], dtype=object)

In [43]:
### Generate the list of operators
'''
Assumes the number of operators is three. This needs to be able to expand and contract base on the 
number of operators inputted
'''
# Pattern of operator based on sorting by SN and DateTime
operator_pattern = np.array([1,1,1, 2,2,2, 3,3,3])

# Create the repeated array
operator_repeated_array = np.tile(operator_pattern, num_parts)

## Tests
#operator_repeated_array
#len(operator_repeated_array)

In [51]:
### Build miniTab dataframe
df_it_Char_TP = df_raw[['DateTime', 'SN', 'CharName', 'TP Measure']]
'''

'''
# Defining dataframe to build for Minitab
df_minitab = pd.DataFrame()
# Adding operator
df_minitab['Operators'] = operator_repeated_array
# Adding trial number
df_minitab['Trials'] = repeated_array
# Selecting one char, then sorting on SN and date, pulling SN values
df_minitab['SN'] = df_it_Char_TP[df_it_Char_TP['CharName'] == all_char[0]].sort_values(['SN','DateTime'],ascending=[True, True])['SN'].values

for char in all_char:
    # Pull the char data
    df_single_char = df_it_Char_TP[df_it_Char_TP['CharName'] == char]
    # Ensure measurements are in chronological order and SN so operator and trials line up
    df_single_char_date_order = df_single_char.sort_values(['SN','DateTime'],ascending=[True, True])
    # Building the miniTab dataframe
    df_minitab[char+'_TP'] = df_single_char_date_order['TP Measure'].values

df_minitab.head(10)

,Operators,Trials,SN,C2_3_TP,C4A_TP,C4B_TP,C4C_TP,C4D_TP,C4E_TP,C4F_TP,...,C27B_TP,C27C_TP,C27D_TP,C31A_TP,C37A_TP,C50A_TP,C31B_TP,C37B_TP,C50B_TP,C52_TP
0,1,1,063,0.283,0.079,0.097,0.105,0.081,0.179,0.135,...,0.890,0.589,1.225,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2,063,0.265,0.089,0.108,0.097,0.084,0.160,0.130,...,0.884,0.589,1.201,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,3,063,0.292,0.068,0.096,0.101,0.085,0.165,0.149,...,0.886,0.563,1.213,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,1,063,0.275,0.082,0.092,0.096,0.087,0.163,0.118,...,0.900,0.604,1.216,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,2,063,0.294,0.059,0.093,0.105,0.089,0.171,0.110,...,0.894,0.589,1.227,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2,3,063,0.313,0.069,0.096,0.113,0.098,0.182,0.128,...,0.898,0.604,1.242,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3,1,063,0.290,0.073,0.094,0.101,0.093,0.170,0.133,...,0.890,0.601,1.236,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,3,2,063,0.302,0.064,0.085,0.110,0.103,0.178,0.128,...,0.908,0.602,1.236,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,3,3,063,0.287,0.088,0.099,0.108,0.090,0.175,0.148,...,0.895,0.612,1.238,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1,1,064,0.152,0.418,0.449,0.426,0.362,0.397,0.294,...,0.614,0.207,0.418,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
### Output dataframe to CSV
# sort specific for miniTab
df_minitab = df_minitab.sort_values(['Operators', 'SN'], ascending=[True, True])

# Output to csv
df_minitab.to_csv('minitab_ready.csv', index=False)

In [46]:
### Test, Build SN Column
# Pull data on one char
# df_it_Char_TP[df_it_Char_TP['CharName'] == all_char[0]].sort_values(['SN','DateTime'],ascending=[True, True])
x = df_it_Char_TP[df_it_Char_TP['CharName'] == all_char[0]].sort_values(['SN','DateTime'],ascending=[True, True])['SN'].values
x

array(['063', '063', '063', '063', '063', '063', '063', '063', '063',
       '064', '064', '064', '064', '064', '064', '064', '064', '064',
       '075', '075', '075', '075', '075', '075', '075', '075', '075',
       '197', '197', '197', '197', '197', '197', '197', '197', '197',
       '206', '206', '206', '206', '206', '206', '206', '206', '206',
       '207', '207', '207', '207', '207', '207', '207', '207', '207',
       '211', '211', '211', '211', '211', '211', '211', '211', '211',
       '212', '212', '212', '212', '212', '212', '212', '212', '212',
       '214', '214', '214', '214', '214', '214', '214', '214', '214',
       '216', '216', '216', '216', '216', '216', '216', '216', '216'],
      dtype=object)

## Misc

In [47]:
### Inital Matrix Contruction

In [48]:
### Creating the column for the operator


In [49]:
### Building upon the above with iterating through the whole data frame

# Pull the different characteristics from CharName, without duplicates
char_names = df_Char_TP['CharName'].unique()
#print(char_names) # test
#print(type(char_names)) # test

# Iterate through by characteristics and generate separate data frames based CharName/TP
    # Have to index through char_names to select the specific character from df_Char_TP
print(char_names[0]) # so use the standard index syntax
    # Select character based on the index
df_single_char = df_Char_TP[df_Char_TP['CharName'] == char_names[0]] # filtering based on the index of char_names
# df_single_char # test
char_range = df_single_char['TP Measure'].max() - df_single_char['TP Measure'].min()
print(f'The range of {char_names[0]} is {char_range}')
    # Iterate
for cname in char_names:
    df_single_char = df_Char_TP[df_Char_TP['CharName'] == cname]
    char_range = df_single_char['TP Measure'].max() - df_single_char['TP Measure'].min()
    print(f'The range of {cname} is {char_range}')



C2_3
The range of C2_3 is 0.28300000000000003
The range of C2_3 is 0.28300000000000003
The range of C4A is 0.548
The range of C4B is 0.40399999999999997
The range of C4C is 0.351
The range of C4D is 0.296
The range of C4E is 0.269
The range of C4F is 0.29100000000000004
The range of C4G is 0.338
The range of C4H is 0.349
The range of C11 is 0.195
The range of C13A is 0.45899999999999996
The range of C13B is 0.6619999999999999
The range of C17A is 0.771
The range of C17B is 0.938
The range of C18A is 0.314
The range of C18B is 1.257
The range of C26A is 1.2730000000000001
The range of C26B is 1.031
The range of C26C is 0.594
The range of C26D is 1.321
The range of C27A is 1.119
The range of C27B is 0.986
The range of C27C is 0.7939999999999999
The range of C27D is 0.8240000000000001
The range of C31A is 0.0
The range of C37A is 0.0
The range of C50A is 0.0
The range of C31B is 0.0
The range of C37B is 0.0
The range of C50B is 0.0
The range of C52 is 0.0


In [50]:
### Cleaning up the above into clean code

# New data frame that only contains CharName and TP Measure
df_Char_TP = df[['CharName', 'TP Measure']]

# Pull the different characteristics from CharName, without duplicates
char_names = df_Char_TP['CharName'].unique() # pulls unique values

# Iterate through by characteristics and generate the ranges
for cname in char_names:
    df_single_char = df_Char_TP[df_Char_TP['CharName'] == cname] # filters based on character
    char_range = df_single_char['TP Measure'].max() - df_single_char['TP Measure'].min() # Calculates the range
    print(f'The range of {cname} is {char_range}') # prints the range


NameError: name 'df' is not defined

To Do:
-test to see if the ranges are accurate
-generate a new csv file or some kind of visualization
-conduct for feature size
-add addition statistics that would be for a gage rr
-in final code create functions to perform the various statistics
-control the number of decimal places on the output

Full program
-enter the gage rr trial.. is it a 1x1x10 or a 3x3x10 ect
    -test the data to confirm selection
-enter the tolerance limits of each characteristic
    -user input or a csv input?
        -maybe after determining the characteristics from the data iterate and ask for each tolerance
            -the idea is to keep the program universal to any part
-spit out results for each characteristic, how? a tab each in excel? In a notebook?

-take the math and create a library, tinyTAB
    - .cpk(data, char, ect, ect)



In [ ]:
### Same as above but for standard deviation

# New data frame that only contains CharName and TP Measure
df_Char_TP = df[['CharName', 'TP Measure']]

# Pull the different characteristics from CharName, without duplicates
char_names = df_Char_TP['CharName'].unique() # pulls unique values

# Iterate through by characteristics and generate the ranges
for cname in char_names:
    df_single_char = df_Char_TP[df_Char_TP['CharName'] == cname] # filters based on character
    char_range = df_single_char['TP Measure'].std() # calculate the standard deviation
    print(f'The standard deviation of {cname} is {char_range}') # prints the range

Resources for Cpk
-https://www.linkedin.com/pulse/calculating-cp-cpk-uma-maheswari-manchala-2c/
-https://www.geeksforgeeks.org/process-capability-index-cpk-formula/



In [ ]:
# Creating a Pandas series
data = [1, 2, 3, 4, 5]
s = pd.Series(data)

# Computing the mean of a Pandas series
mean = s.mean()
print(mean)



## Thee End